In [40]:
import os
import re
import yfinance as yf
import PyPDF2
from textblob import TextBlob
import pandas as pd
import datetime
from textblob import TextBlob
import datefinder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import nltk

In [41]:
###get the historical stock data of netflix
# Get Netflix's stock data
nflx = yf.Ticker("NFLX")

# Define the start and end dates
start_date = "2013-06-17"

# Get the historical market data for the specified date range
hist = nflx.history(start=start_date)



In [42]:
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2013-06-17 00:00:00-04:00,32.220001,32.928570,31.964287,32.747143,35851200,0.0,0.0
2013-06-18 00:00:00-04:00,32.891430,33.000000,32.342857,32.689999,17476900,0.0,0.0
2013-06-19 00:00:00-04:00,32.571430,33.697144,32.515713,33.187141,22353100,0.0,0.0
2013-06-20 00:00:00-04:00,32.781429,33.000000,31.505714,31.931429,21116200,0.0,0.0
2013-06-21 00:00:00-04:00,32.150002,32.340000,30.600000,30.985714,23398200,0.0,0.0


In [43]:
# Reset the index to make the date a column
hist.reset_index(inplace=True)

# Convert the date to the desired format (without timezone)
hist['Date'] = hist['Date'].dt.strftime('%Y-%m-%d')

# Drop unwanted columns
hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

####the reason for doing this process is to realise the following dates match operations

In [44]:
# Write the data to a CSV file
NetflixStock = 'netflix_stock_data.csv'
hist.to_csv(NetflixStock)

print(f"Data saved to {NetflixStock}")


Data saved to netflix_stock_data.csv


In [45]:

# Function to extract text from a PDF file
def extract_text_from_pdf(financial_reports):
    pdf_file_obj = open(financial_reports, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page_obj = pdf_reader.pages[page_num]
        text += page_obj.extract_text()
    pdf_file_obj.close()
    return text

In [46]:

# Get a list of all PDF files in the directory
directory = "/Users/rachel_wang/Desktop/arp/financial_reports/"
files = [f for f in os.listdir(directory) if f.endswith(".pdf")]

# Process each file
for file in files:
    # Extract text
    text = extract_text_from_pdf(os.path.join(directory, file))
    
    # Save the extracted text to a new .txt file
    with open(os.path.join(directory, f"{file[:-4]}.txt"), 'w') as f:
        f.write(text)



In [47]:
# Get a list of all txt files in the directory
directorytxt = "/Users/rachel_wang/Desktop/arp/financial_reports/"
shareholder_files = [f for f in os.listdir(directorytxt) if f.endswith(".txt")]

def format_date(date_string):
    # Check if the date string matches the pattern "MonthDay,Year"
    match = re.match(r'([A-Za-z]+)(\d{1,2}), (\d{4})', date_string)
    if match:
        month = match.group(1)
        day = match.group(2)
        year = match.group(3)
        return f"{month} {day}, {year}"
    return date_string

# Extract release dates and content from shareholder letters
shareholder_data = []

for file_path in shareholder_files:
    full_path = os.path.join(directorytxt, file_path)
    with open(full_path, 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()

        # Extracting year and quarter from the file name
        year_quarter_match = re.search(r'Q\d{1}-\d{2}', file_path)
        if year_quarter_match:
            year_quarter = year_quarter_match.group()
            year = "20" + year_quarter.split('-')[1]
            quarter = year_quarter.split('-')[0]
        else:
            year, quarter = None, None

        # Pattern to match potential date strings
        date_pattern = r'([A-Za-z]+\d{1,2}, \d{4})'

        # Assuming the release date is the first date mentioned in the file
        date_match = re.search(date_pattern, content)
        
        # Inside your loop for each file:
        matches = datefinder.find_dates(content)
        first_mentioned_date = next(matches, None)  # Get the first date found

        if first_mentioned_date:
            first_mentioned_date = first_mentioned_date.strftime('%B %d, %Y')
        else:
            first_mentioned_date = None

        shareholder_data.append({
            "year": year,
            "quarter": quarter,
            "release_date": first_mentioned_date,
            "content": content
        })

shareholder_data


[{'year': '2023',
  'quarter': 'Q2',
  'release_date': 'July 19, 2023',
  'content': 'July19,2023\nSummary:\n●InMay,wesuccessfullylaunchedpaidsharingin100+countries,representingmorethan80%ofourrevenuebase.\n○Revenueineachregionisnowhigherthanpre-launch,withsign-upsalreadyexceedingcancellations.\n○Paidnetadditionswere5.9MinQ2,andtodaywe’rerollingoutpaidsharingtoalmostalloftheremainingcountries.\n●Q2‘23revenueof$8.2Bandoperatingprofitof$1.8Bweregenerallyin-linewithourforecast—andweexpectrevenuegrowthtoaccelerateinthesecondhalfof‘23aswestarttoseethefullbenefitsofpaidsharingpluscontinuedsteadygrowthinourad-supportedplan.We’restilltargetingafullyear2023operatingmarginof18%to20%.\n●We’realeaderintermsofstreamingengagementand,perNielsen,wehadthetoporiginalstreamingseriesintheUSfor24ofthefirst25weeksof2023,andthetopmoviefor21weeks.\nWhilewe’vemadesteadyprogressthisyear,wehavemoreworktodotoreaccelerateourgrowth.Weremainfocusedon:creatingasteadydrumbeatofmustwatchshowsandmovies;improvingmonetiza

In [48]:

# Load the stock prices from the CSV file
stock_data = pd.read_csv("netflix_stock_data.csv", parse_dates=["Date"])

# Function to extract the stock prices for the next five days after the release date
def get_stock_data_after_release(release_date):
    try:
        release_date = datetime.datetime.strptime(release_date, '%B %d, %Y')
    except:
        return None, None, None, None

    # Filter data for the next 5 days
    mask = (stock_data['Date'] > release_date) & (stock_data['Date'] <= release_date + datetime.timedelta(days=5))
    five_days_data = stock_data[mask]

    if not five_days_data.empty:
        first_open = five_days_data.iloc[0]['Open']
        first_close = five_days_data.iloc[0]['Close']
        last_close = five_days_data.iloc[-1]['Close']
        highest_price = five_days_data['High'].max()
        lowest_price = five_days_data['Low'].min()
        return first_open, first_close, last_close, highest_price, lowest_price
    else:
        return None, None, None, None, None

# Extract the relevant stock data for each shareholder letter
for data in shareholder_data:
    data['first_open'], data['first_close'], data['last_close'], data['highest_price'], data['lowest_price'] = get_stock_data_after_release(data['release_date'])

shareholder_data


[{'year': '2023',
  'quarter': 'Q2',
  'release_date': 'July 19, 2023',
  'content': 'July19,2023\nSummary:\n●InMay,wesuccessfullylaunchedpaidsharingin100+countries,representingmorethan80%ofourrevenuebase.\n○Revenueineachregionisnowhigherthanpre-launch,withsign-upsalreadyexceedingcancellations.\n○Paidnetadditionswere5.9MinQ2,andtodaywe’rerollingoutpaidsharingtoalmostalloftheremainingcountries.\n●Q2‘23revenueof$8.2Bandoperatingprofitof$1.8Bweregenerallyin-linewithourforecast—andweexpectrevenuegrowthtoaccelerateinthesecondhalfof‘23aswestarttoseethefullbenefitsofpaidsharingpluscontinuedsteadygrowthinourad-supportedplan.We’restilltargetingafullyear2023operatingmarginof18%to20%.\n●We’realeaderintermsofstreamingengagementand,perNielsen,wehadthetoporiginalstreamingseriesintheUSfor24ofthefirst25weeksof2023,andthetopmoviefor21weeks.\nWhilewe’vemadesteadyprogressthisyear,wehavemoreworktodotoreaccelerateourgrowth.Weremainfocusedon:creatingasteadydrumbeatofmustwatchshowsandmovies;improvingmonetiza

In [49]:
# Convert to DataFrame
shareholder_df = pd.DataFrame(shareholder_data)

# Write to CSV
csv_path = "shareholder_content_price.csv"
shareholder_df.to_csv(csv_path, index=False)

In [50]:
####this is a very basic try########
####################################
# Extract sentiment scores from the content of each shareholder letter
for data in shareholder_data:
    content = data["content"]
    blob = TextBlob(content)
    sentiment_score = blob.sentiment.polarity
    data["sentiment_score"] = sentiment_score


In [51]:

# Extract relevant data for new CSV
csv_data = []
for data in shareholder_data:
    csv_data.append({
        "year": data["year"],
        "quarter": data["quarter"],
        "release_date": data["release_date"],
        "sentiment_score": data["sentiment_score"],
        "first_open": data["first_open"],
        "first_close": data["first_close"],
        "last_close": data["last_close"],
        "highest_price": data["highest_price"],
        "lowest_price": data["lowest_price"]
    })

# Convert to DataFrame
df_shareholder = pd.DataFrame(csv_data)

# Write to CSV
csv_path = "Netflix_stock_letter_base.csv"
df_shareholder.to_csv(csv_path, index=False)

csv_path

'Netflix_stock_letter_base.csv'

In [52]:
# Removing rows with NaN values
# df_shareholder_cleaned = df_shareholder.dropna()
df_shareholder_cleaned = df_shareholder

# Splitting the cleaned data into training and test sets
X_cleaned = df_shareholder_cleaned[['sentiment_score', 'first_open']]
y_cleaned = df_shareholder_cleaned['first_close']

X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.3, random_state=42)

# Training the linear regression model on the cleaned data
model_cleaned = LinearRegression()
model_cleaned.fit(X_train_cleaned, y_train_cleaned)

# Predicting on the test set
y_pred_cleaned = model_cleaned.predict(X_test_cleaned)

# Calculating the Mean Squared Error (MSE) of the model
mse_cleaned = mean_squared_error(y_test_cleaned, y_pred_cleaned)

mse_cleaned


91.69114250408427

In [53]:
y_pred_cleaned

array([440.00618505, 245.76630795, 324.7975545 , 178.08705911,
       305.50081372, 257.59813701, 337.9939821 ])

In [54]:
y_test_cleaned

0     437.420013
17    226.190002
15    325.209991
1     183.600006
8     293.350006
5     250.289993
11    342.500000
Name: first_close, dtype: float64

In [55]:
X_test_cleaned

,sentiment_score,first_open
0,-0.029167,447.000000
17,0.107184,245.199997
15,0.126166,323.760010
1,0.128802,176.119995
8,0.123386,304.489990
5,0.150145,255.050003
11,0.131178,336.790009


In [56]:
# Importing necessary libraries for text processing
from bs4 import BeautifulSoup
import re
shareholder_path = "/Users/rachel_wang/Desktop/shareholder_content_price.csv"
sahreholder_df = pd.read_csv(shareholder_path)

# Function to preprocess the content
def preprocess_text(text):
    # Removing HTML tags if any
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Removing extra whitespaces, line breaks, and other non-text content
    text = re.sub(r'\n', ' ', text) # Replacing line breaks with spaces
    text = re.sub(r'\s+', ' ', text) # Replacing multiple spaces with a single space
    text = text.strip() # Removing leading and trailing spaces
    
    return text

# Applying the preprocess_text function to the "content" column
shareholder_df['content'] = shareholder_df['content'].apply(preprocess_text)

# Showing the first few rows after preprocessing
shareholder_df.head()


,year,quarter,release_date,content,first_open,first_close,last_close,highest_price,lowest_price
0,2023,Q2,"July 19, 2023","July19,2023 Summary: ●InMay,wesuccessfullylaun...",447.000000,437.420013,428.369995,448.500000,419.190002
1,None,None,"July 17, 2017","July 17, 2017 Fellow shareholders For Q2, reve...",176.119995,183.600006,188.539993,191.500000,174.240005
2,2021,Q2,"July 20, 2021","July 20, 2021 Fellow shareholders, In Q2, reve...",526.130005,513.630005,515.409973,530.989990,504.660004
3,None,None,"October 16, 2017","October 16, 2017 Fellow shareholders, We are g...",200.110001,199.479996,194.160004,204.380005,191.160004
4,2022,Q2,"July 19, 2022","July 19, 2022 Fellow shareholders, Q2 was bett...",208.169998,216.440002,220.440002,229.350006,200.100006


Try vader sentiment analysis

In [57]:
vader_sentimentscore = pd.DataFrame()
# Importing the VADER sentiment analysis tool from NLTK
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initializing the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to get the sentiment score
def get_sentiment_score(text):
    sentiment_score = sia.polarity_scores(text)
    return sentiment_score['compound']

# Applying the get_sentiment_score function to the "content" column and storing the results in a new column
vader_sentimentscore['sentiment_score'] = shareholder_df['content'].apply(get_sentiment_score)

# Showing the first few rows with the new sentiment score column
vader_sentimentscore.head()


,sentiment_score
0,0.8977
1,0.9999
2,0.9998
3,0.9999
4,0.9999


Try LM

In [58]:
# Loading the Loughran and McDonald (LM) Lexicon CSV file
lm_lexicon_path = 'Loughran-McDonald_MasterDictionary_1993-2021.csv'
lm_lexicon = pd.read_csv(lm_lexicon_path)

# Displaying the first few rows of the LM Lexicon
lm_lexicon.head()

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Syllables,Source
0,AARDVARK,1,354,1.550080e-08,1.422600e-08,3.815486e-06,99,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.313627e-10,8.653817e-12,9.241714e-09,1,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,3.940882e-10,1.169679e-10,5.290465e-08,7,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,29,1.269840e-09,6.654735e-10,1.595100e-07,28,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8570,3.752595e-07,3.809464e-07,3.529356e-05,1108,0,0,0,0,0,0,0,3,12of12inf


In [59]:
# Extracting positive and negative words from the LM Lexicon
positive_words_lm = lm_lexicon[lm_lexicon['Positive'] > 0]['Word'].str.lower().tolist()
negative_words_lm = lm_lexicon[lm_lexicon['Negative'] > 0]['Word'].str.lower().tolist()

# Summary of the positive and negative words count
len(positive_words_lm), len(negative_words_lm)


(347, 2345)

In [60]:
lm_analysis = pd.DataFrame()
# Function to calculate sentiment score using the LM Lexicon
def calculate_lm_sentiment_score(text):
    # Tokenizing the text into words
    words = text.lower().split()
    
    # Counting occurrences of positive and negative words
    positive_count = sum(word in positive_words_lm for word in words)
    negative_count = sum(word in negative_words_lm for word in words)
    
    # Calculating sentiment score as the difference between positive and negative counts
    sentiment_score = positive_count - negative_count
    
    return sentiment_score

# Applying the calculate_lm_sentiment_score function to the "content" column
lm_analysis['lm_sentiment_score'] = shareholder_df['content'].apply(calculate_lm_sentiment_score)

# Showing the first few rows with the new LM sentiment score column
lm_analysis.head()


,lm_sentiment_score
0,-2
1,18
2,27
3,31
4,30


In [61]:
lm_analysis['date']= shareholder_df['release_date']
lm_analysis['first_open']= shareholder_df['first_open']
lm_analysis['first_close']= shareholder_df['first_close']
lm_analysis.head()

,lm_sentiment_score,date,first_open,first_close
0,-2,"July 19, 2023",447.000000,437.420013
1,18,"July 17, 2017",176.119995,183.600006
2,27,"July 20, 2021",526.130005,513.630005
3,31,"October 16, 2017",200.110001,199.479996
4,30,"July 19, 2022",208.169998,216.440002


In [63]:

# Splitting the cleaned data into training and test sets
X_lm = lm_analysis[['lm_sentiment_score', 'first_open']]
y_lm = lm_analysis['first_close']

X_train, X_test, y_train, y_test= train_test_split(X_lm, y_lm, test_size=0.3, random_state=42)

# Training the linear regression model on the cleaned data
model_lm = LinearRegression()
model_lm.fit(X_train, y_train)

# Predicting on the test set
y_pred_lm = model_lm.predict(X_test)

# Calculating the Mean Squared Error (MSE) of the model
mse_lm = mean_squared_error(y_test, y_pred_lm)

mse_lm

90.31546548121337

In [64]:
y_pred_lm

array([445.13611737, 244.68305291, 322.73124837, 175.61764516,
       304.12744224, 254.25427766, 337.11983747])

In [65]:
y_test

0     437.420013
17    226.190002
15    325.209991
1     183.600006
8     293.350006
5     250.289993
11    342.500000
Name: first_close, dtype: float64

In [66]:
X_test

,lm_sentiment_score,first_open
0,-2,447.000000
17,22,245.199997
15,13,323.760010
1,18,176.119995
8,30,304.489990
5,15,255.050003
11,51,336.790009
